TP ia puissance 4

Puissance 4
6 lignes / 7 colonnes
Récompenses
pion au hasard -> - 0.1
win = +1
lose = -1

In [156]:
from prettytable import PrettyTable
import pandas as pd
import numpy as np
import random

In [157]:
class Board:
    def __init__(self, player1, player2, line_number, columns_number, q_table=None):
        
        self.columns_number = columns_number
        self.line_number = line_number
        
        self.table = [['' for _ in range(columns_number)] for _ in range(line_number)]
        
        self.player1 = player1
        self.player2 = player2
        
        if q_table is None:
            headers = ['state'] + [f'col {str(x)}' for x in range(self.columns_number)]
            self.q_table = pd.DataFrame(columns=headers)
            self.initialize_state()
        else:
            self.q_table = q_table
            
    def is_draw(self):
        return all(self.table[0][i] != '' for i in range(self.columns_number))
        
            
                   
    def play_in_column(self, player, column):
        print('player ', player.symbol, 'plays in column ', column)
        if column < 0 or column >= self.columns_number:
            raise ValueError("Invalid column number")
        
        if self.check_if_column_is_full(column):
            raise ValueError("Column is full")
        
        # Je récupère la ligne la plus basse possible ou la colonne est vide
        for line in range(self.line_number-1, -1, -1):
            if self.table[line][column] == '':
                self.table[line][column] = player.symbol
                break
        return self.get_string_representation()
            
            
    def get_q_table(self, column_number):
    # renvoie la q table pour un state donné
        state = self.get_string_representation()
        print('---------------------')
        print('state = ', state)
        if state in self.q_table.index and column_number < len(self.q_table.columns):
            ligne = self.q_table.loc[state]
            value = ligne[column_number]
            return ligne
        else:
            print("The state or column doesn't exist in the Q table.")
            return None
        
    def get_q_value(self, state, column_number=2):
        if state in self.q_table.index and column_number < len(self.q_table.columns):
            value = self.q_table.loc[state, column_number]
            return value
        else:
            print("The state or column doesn't exist in the Q table.")
            return None  
        
        
        
    def get_possible_actions(self):
        # Retourne la liste des colonnes où l'on peut encore jouer
        list_of_columns_availbe_to_play = []
        
        for column in range(self.columns_number):
            if(self.check_if_column_is_full(column) == False):
                list_of_columns_availbe_to_play.append(column)
        return list_of_columns_availbe_to_play
    
    
    def get_max_actions_possible(self):
        # Renvoie le numéro de la colonne où l'on peut jouer pour maximiser les points
        actions_possibles = self.get_possible_actions()
        list_of_actions = {}

        for action in actions_possibles:
            actions_points = self.get_q_table(action)
            list_of_actions[action] = actions_points
        return max(list_of_actions, key=list_of_actions.get)
            
    
    def check_if_column_is_full(self, column_number):
        # Retourne True si la colonne est pleine, False sinon
        if column_number < 0 or column_number >= self.columns_number:
            raise ValueError("Invalid column number")
        return self.table[0][column_number] != ''
            
    
    def print_board(self):
        tab = PrettyTable()
        tab.header = False
        for line in self.table:
            tab.add_row([x for x in line])
        print(tab)
        
        
    def print_q_table(self):
        print(self.q_table.head())
        
        
    def initialize_state(self):
        # Ajoute l'état du state dans la q table
        table_state = self.get_string_representation()
        if table_state not in self.q_table.index:
            
            # print('number of zeros = ', np.zeros(self.columns_number))
            # print('df = ', self.q_table)
            new_row = [table_state] + [0]*self.columns_number
            # print("new row = ", new_row)

            self.q_table.loc[table_state] = new_row
            
            # print('q table = ', self.q_table)
            
            # new_row = pd.Series([0]*self.columns_number, name=table_state):
            # self.q_table = self.q_table.append(new_row)
            
    def update_q_table(self, old_state, action, reward, new_state):
        print("indexes")
        print(self.q_table.index)
        if old_state not in self.q_table.index:
            print('old state not in index')
            self.q_table.loc[old_state] = [0]*len(self.q_table.columns)
        if new_state not in self.q_table.index:
            print('new state not in index')
            self.q_table.loc[new_state] = [0]*len(self.q_table.columns)
            
        print('checked old and new states')
        print(self.q_table.index)
        
        print(self.q_table)
        old_q_value = self.q_table.loc[old_state, action]
        max_new_q_value = self.q_table.loc[new_state].max()
        
        # Equation de Bellman
        new_q_value = old_q_value + self.learning_rate * (reward + self.discount_factor * max_new_q_value - old_q_value)
        
        # Mettre à jour la valeur Q
        print("ici ça tourne")
        self.q_table.loc[old_state, action] = new_q_value    
        print("ici pas sur que ça tourne")    
        
    def get_string_representation(self):
        return str(self.table)
    
    def is_winning_move(self, column):
        # Trouver la ligne de la pièce la plus haute dans la colonne
        for line in range(self.line_number):
            if self.table[line][column] != '':
                break
        else:
            raise ValueError("La colonne est vide")

        # Vérifie les conditions de victoire (4 en ligne)
        piece = self.table[line][column]
        # Check horizontal, vertical, and diagonal directions
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        for dr, dc in directions:
            count = 1
            for i in range(1, 4):
                r, c = line + dr*i, column + dc*i
                if 0 <= r < self.line_number and 0 <= c < self.columns_number and self.table[r][c] == piece:
                    count += 1
                else:
                    break
            for i in range(1, 4):
                r, c = line - dr*i, column - dc*i
                if 0 <= r < self.line_number and 0 <= c < self.columns_number and self.table[r][c] == piece:
                    count += 1
                else:
                    break
            if count >= 4:
                return True
        return False
    
    
    def train_ai(self, epoch, epsilon = 1, min_epsilon = 0.01, epsilon_decay = 0.0001):
        
        for e in range(0, epoch+1):
            
            # On entraine l'ia contre elle même
            for i in range(1, 3):
                if i == 1:
                    player = self.player1
                else:
                    player = self.player2            
                #Exporation ou connaissance ?
                if random.uniform(0, 1) < epsilon:
                    A = random.choice(self.get_possible_actions())
                    print('E', e, "Exp")
                else:
                    # actions_possibles = self.get_possible_actions()
                    A = self.get_max_actions_possible()
                    
                if player == self.player1:
                    old_state = self.get_string_representation()
                    next_state = self.play_in_column(player, A)
                    if self.is_winning_move(A):
                        reward = 1
                    else: reward = -0.01
                        # Je dois update la q table
                    print('old state = ', old_state)
                    print('next state = ', next_state)
                    print('A = ', A)
                    self.update_q_table(old_state, A, reward, next_state)
                    if reward == 1:
                        print(player.symbol, "a gagné")
                        break
                
                    # :je fais un else pr savoir si c'est match nul
                        
                    # Je check si il a gagné
                    # Si oui je donne une reward
                    # Si non je donne rien ou alors -0.01/0.1
                
            
            self.print_board()
            



In [158]:
class Player:
    def __init__(self, symbol):
        self.symbol = symbol

    def __str__(self):
        return f'{self.name} ({self.symbol})'

    def __repr__(self):
        return self.__str__()

In [159]:
player1 = Player('X')
player2 = Player('O')
board1 = Board(player1=player1, player2=player2, line_number=6, columns_number=7)

# board1.print_board()
# board1.get_possible_actions()
# board1.initialize_state()
# board1.print_q_table()
# board1.play_in_column(player1, 0)
# board1.print_board()
# board1.play_in_column(player2, 0)
# board1.play_in_column(player1, 1)
# board1.play_in_column(player2, 0)
# board1.print_board()
# board1.initialize_state()
# board1.print_q_table()



In [160]:
# Au début c'est soit le joueur soi l'ia qui commence
# On place un pion sur la ligne du bas forcément
# Je vais créer une classe pour le plateau


# mon etat c est pas seulement les coordonnées du pion  mais c'est tout le plateau
# avec sur chaque case la valeur associe
# Donc la q table va etre groooooosse
board2 = Board(player1=player1, player2=player2, line_number=6, columns_number=7)
# print(board2.get_q_table(2))
print('-----------')
print(board2.get_q_value(board2.get_string_representation(), 2))
# board2.train_ai(10)
# board2.print_board()

---------------------
state =  [['', '', '', '', '', '', ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', '']]
state    [['', '', '', '', '', '', ''], ['', '', '', ''...
col 0                                                    0
col 1                                                    0
col 2                                                    0
col 3                                                    0
col 4                                                    0
col 5                                                    0
col 6                                                    0
Name: [['', '', '', '', '', '', ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', '']], dtype: object
-----------
                                                                                                state 

C:\Users\flori\AppData\Local\Temp\ipykernel_40948\2117961671.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = ligne[column_number]
